**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [6]:
!pip install keras-flops

In [39]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops
from keras.datasets import cifar100
from sklearn.preprocessing import OneHotEncoder

In [8]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [35]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = cifar100.load_data(label_mode='fine')

In [42]:
# Преобразуйте метки классов в one_hot формат
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_val = tf.keras.utils.to_categorical(y_val, CLASSES)

Проверяю как закодировалось

In [52]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [45]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [53]:
input_shape = (32, 32, 3)

In [176]:
model = tf.keras.models.Sequential(
[
    tf.keras.layers.Conv2D(30, kernel_size=(3, 3), activation='relu', input_shape=input_shape, strides=(2,2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    
    tf.keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(CLASSES, activation='softmax')
])

In [174]:
model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_64 (Conv2D)           (None, 16, 16, 30)        1470      
_________________________________________________________________
batch_normalization_94 (Batc (None, 16, 16, 30)        120       
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 8, 8, 30)          0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 6, 6, 8)           2168      
_________________________________________________________________
batch_normalization_95 (Batc (None, 6, 6, 8)           32        
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 3, 3, 8)           0         
_________________________________________________________________
flatten_39 (Flatten)         (None, 72)              

In [177]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-25 20:59:34.479801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 20:59:34.480201: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-25 20:59:34.480330: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-25 20:59:34.480863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 20:59:34.481350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 20:59:34.481727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

_101/batchnorm/mul_2 (256/256 flops)
  sequential_40/batch_normalization_101/batchnorm/mul_1 (256/256 flops)
  sequential_40/batch_normalization_101/batchnorm/mul (256/256 flops)
  sequential_40/batch_normalization_101/batchnorm/add_1 (256/256 flops)
  sequential_40/batch_normalization_101/batchnorm/add (256/256 flops)
  sequential_40/batch_normalization_100/batchnorm/sub (256/256 flops)
  sequential_40/batch_normalization_100/batchnorm/mul_2 (256/256 flops)
  sequential_40/batch_normalization_100/batchnorm/mul_1 (256/256 flops)
  sequential_40/batch_normalization_100/batchnorm/mul (256/256 flops)
  sequential_40/batch_normalization_100/batchnorm/add_1 (256/256 flops)
  sequential_40/batch_normalization_100/batchnorm/add (256/256 flops)
  sequential_40/dense_68/BiasAdd (100/100 flops)

======================End of Report==========================


In [180]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [184]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=256
)

Epoch 1/256
391/391 [==============================] - 3s 6ms/step - loss: 3.7619 - accuracy: 0.1382 - val_loss: 3.9609 - val_accuracy: 0.1406
Epoch 2/256
391/391 [==============================] - 2s 5ms/step - loss: 3.2002 - accuracy: 0.2209 - val_loss: 3.2812 - val_accuracy: 0.2231
Epoch 3/256
391/391 [==============================] - 2s 5ms/step - loss: 3.0084 - accuracy: 0.2591 - val_loss: 2.9673 - val_accuracy: 0.2746
Epoch 4/256
391/391 [==============================] - 2s 5ms/step - loss: 2.8858 - accuracy: 0.2821 - val_loss: 2.9264 - val_accuracy: 0.2818
Epoch 5/256
391/391 [==============================] - 2s 6ms/step - loss: 2.7920 - accuracy: 0.3015 - val_loss: 2.7936 - val_accuracy: 0.3095
Epoch 6/256
391/391 [==============================] - 2s 5ms/step - loss: 2.7231 - accuracy: 0.3139 - val_loss: 2.7356 - val_accuracy: 0.3232
Epoch 7/256
391/391 [==============================] - 2s 5ms/step - loss: 2.6672 - accuracy: 0.3267 - val_loss: 2.7167 - val_accuracy: 0.3276

KeyboardInterrupt: 